In [1]:
import pandas as pd
import implicit
import lightfm
import scipy

import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split

c:\study\venv\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [4]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("en")

def preprocess_txt(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [6]:
# Загрузим рейтинги, также как делали
ratings = pd.read_csv("./data/ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user id', 'item id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)

ratings['score'] = (ratings['rating'] > 2).apply(int)
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

ratings.head()

,user id,item id,rating,timestamp,score
214,259,255,4,874724710,1
83965,259,286,4,874724727,1
43027,259,298,4,874724754,1
21396,259,185,4,874724781,1
82655,259,173,4,874724843,1


In [7]:
# Загрузим данные о фильмах, сразу же создадим колонку с title+description предобработанные для обучения fasttext
movies = pd.read_csv("./data/films.csv")
movies['joined'] = (movies['title'] + " " + movies["description"]).apply(preprocess_txt)

In [8]:
movies.head()

,id,title,tagline,description,genres,keywords,date,collection,runtime,revenue,...,cast,production_companies,production_countries,popularity,average_vote,num_votes,language,imdb_id,poster_url,joined
0,862,Toy Story,NaN,"Led by Woody, Andy's toys live happily in his ...","animation, comedy, family","jealousy, toy, boy, friendship, friends, rival...",1995-10-30,Toy Story Collection,81.0,373554033.0,...,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",Pixar Animation Studios,United States of America,21.946943,7.7,5415.0,en,tt0114709,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[toy, story, led, woody, andys, toys, live, ha..."
1,8844,Jumanji,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"adventure, fantasy, family","board game, disappearance, based on children's...",1995-12-15,NaN,104.0,262797249.0,...,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","TriStar Pictures, Teitler Film, Interscope Com...",United States of America,17.015539,6.9,2413.0,en,tt0113497,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[jumanji, siblings, judy, peter, discover, enc..."
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"romance, comedy","fishing, best friend, duringcreditsstinger, ol...",1995-12-22,Grumpy Old Men Collection,101.0,0.0,...,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Warner Bros., Lancaster Gate",United States of America,11.712900,6.5,92.0,en,tt0113228,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[grumpier, old, men, family, wedding, reignite..."
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","comedy, drama, romance","based on novel, interracial relationship, sing...",1995-12-22,NaN,127.0,81452156.0,...,"Whitney Houston, Angela Bassett, Loretta Devin...",Twentieth Century Fox Film Corporation,United States of America,3.859495,6.1,34.0,en,tt0114885,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[waiting, exhale, cheated, mistreated, stepped..."
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,comedy,"baby, midlife crisis, confidence, aging, daugh...",1995-02-10,Father of the Bride Collection,106.0,76578911.0,...,"Steve Martin, Diane Keaton, Martin Short, Kimb...","Sandollar Productions, Touchstone Pictures",United States of America,8.387519,5.7,173.0,en,tt0113041,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[father, bride, part, ii, just, george, banks,..."


In [9]:
# Подготовим кандидатогенератор, который будет отдавать фильмы похожие по текстовому описанию на те, которые оенил пользователь
# Обучим Fasttext и заэмбедим фильмы
sentences = [i for i in movies['joined'] if len(i) > 2]
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

In [10]:
modelFT

In [11]:
# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_movies = annoy.AnnoyIndex(20 ,'angular')

# Будем хранить соответствия не только id-> фильм, но и фильм-> id, чтобы быстрее находить эмбеддинги айтемов
index_map_movies = {}
reverse_index_map = {}
counter = 0

for i in tqdm_notebook(range(len(movies))):
    n_ft = 0
    index_map_movies[counter] = movies.loc[i, "title"]
    reverse_index_map[movies.loc[i, "id"]] = counter
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in movies.loc[i, "joined"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_movies.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_movies.build(10)

<ipython-input-11-8cda0413491a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(movies))):


  0%|          | 0/46628 [00:00<?, ?it/s]

True

In [15]:
# Создадим метод, который по id фильма будет возвращать похожие на него по описанию фильмы
def recommend(movie_id, num_sampled=10):
    if movie_id not in reverse_index_map:
        return []
    return ft_index_movies.get_nns_by_item(reverse_index_map[movie_id], num_sampled)

In [16]:
# Сделаем сразу рандомный кандидатогенератор (он нам пригодится для кандидатогенерации по жанрам)
def sample_random(sample_set, num_sampled=10):
    return np.random.choice(sample_set, num_sampled, replace=False)

In [18]:
# Сделаем кандидатогенератор по последним просмотренным жанрам

movies2genres = {}
genres_dict = {}

for i in range(len(movies)):
    genres = str(movies.loc[i, "genres"]).strip().split(",")
    movies2genres[movies.loc[i, 'id']] = genres
    for genre in genres:
        if genre not in genres_dict:
            genres_dict[genre] = []
        genres_dict[genre].append(movies.loc[i, 'id'])

In [21]:
# Создадим метод, который по последнему пролайканному пользователем фильму будет возвращать набор кандидатов
# КОнечно, целесообразно включать более полную историю, но все-таки это учебный пример

movies_unique = movies['id'].unique()

def candidate_generator(movie_id):
    candidates_ft = []
    if movie_id is not None:
        candidates_ft = list(recommend(movie_id))
    candidates_random = list(sample_random(movies_unique))   


    candidates_genres = []
    for genre in movies2genres.get(movie_id, []):
        candidates_genres += list(sample_random(genres_dict[genre], 5))
    # Отфильтруем входной фильмы (он мог попасть в кандидаты) и уберем дубликаты
    return set([i for i in candidates_ft + candidates_genres + candidates_random if i!= movie_id])

In [23]:
# Проверим как работает наш кандидатогенератор

# candidate_generator(802)

In [26]:
# Подготовим наши ранжирующие факторизационные машины
train_pivot = (1 + ratings.pivot(index="user id", columns="item id", values="score")).fillna(0)
train_pivot.reset_index(inplace=True, drop=False)
train_pivot = scipy.sparse.csr_matrix(train_pivot)

train_pivot

<943x1683 sparse matrix of type '<class 'numpy.float64'>'
	with 100943 stored elements in Compressed Sparse Row format>

In [27]:
model = lightfm.LightFM(loss='warp')
model.fit(train_pivot, epochs=30)

In [38]:
# Посмотрим, насколько статичные рекомендации будут хороши: возьмем последний лайк для каждого пользователя в тренировочном сете
item_ids = set(train['item id'].unique())
known_likes = train[train['score'] > 0].groupby('user id')['item id'].unique().to_dict()

# Получаем последние лайки
last_like = train[train['score'] > 0].drop_duplicates(['user id'], keep="last")[["user id", "item id"]].to_dict()
test_grouped = test[test['score'] > 0].groupby('user id')['item id'].unique().to_dict()


In [52]:
for user_id, liked in tqdm_notebook(test_grouped.items()):
    values = set(liked)
    train_likes = set(known_likes.get(user_id, []))
    print(train_likes)

<ipython-input-52-b63323d4a6ae>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id, liked in tqdm_notebook(test_grouped.items()):


  0%|          | 0/301 [00:00<?, ?it/s]

{1, 2, 3, 4, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 30, 31, 32, 33, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97, 98, 99, 100, 106, 107, 108, 109, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 127, 128, 129, 130, 132, 133, 134, 135, 136, 137, 139, 141, 144, 146, 147, 150, 151, 152, 153, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 214, 215, 216, 217, 218, 220, 221, 222, 223, 224, 226, 227, 228, 229, 230, 232, 234, 235, 236, 238, 239, 240, 241, 246, 248, 249, 250, 251, 253, 257, 258, 262, 265, 267, 268, 269, 270, 272}
{354, 355, 299, 268,

In [60]:
map_at10 = 0

for user_id, liked in tqdm_notebook(test_grouped.items()):
    values = set(liked)
    train_likes = set(known_likes.get(user_id, []))
    candidates = list(candidate_generator(last_like.get(user_id, None)))
    
    # Факторизационные машины не умеют работать с айтемами, которых они не видели
    # ПОэтому надо отфильтровать кандидатов
    candidates = [i for i in candidates if i in item_ids]
    
    # Если мы не смогли ничего отскорить, то считаем, что пользователь просто получил нашу выдачу в рандомном порядке
    recommendations = candidates[:min(10, len(candidates))]
    if len(candidates) != 0:
        recommendations = sorted(list(zip(item_ids, model.predict(user_id, candidates))), key=lambda x: x[1], reverse=True)
        
        recommendations = set([i[0] for i in recommendations if i not in train_likes][:10])  
        
        print(values.intersection(recommendations))
        print(values)
        print(recommendations)
    map_at10 += len(values.intersection(recommendations)) / 10
    
map_at10 = map_at10 / len(test_grouped)
print(map_at10)

<ipython-input-60-9aea1480c34e>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id, liked in tqdm_notebook(test_grouped.items()):


  0%|          | 0/301 [00:00<?, ?it/s]

set()
{320, 321, 322, 260, 327, 328, 329, 331, 340, 181, 346, 347, 348, 318, 351}
{1}
set()
{258, 260, 264, 11, 271, 288, 294, 300, 301, 303, 50, 324, 327, 328, 329, 210, 354, 356, 357, 359, 360, 361, 362}
{1}
set()
{257}
{1}
set()
{258, 900, 269, 270, 288, 294, 937, 300, 301, 302, 306, 313, 315, 319, 333, 339, 345, 347, 352, 748}
{1}
{1}
{1, 514, 516, 518, 135, 265, 1039, 152, 153, 156, 28, 286, 31, 414, 168, 170, 173, 174, 175, 430, 50, 435, 180, 181, 56, 313, 58, 188, 318, 191, 194, 195, 196, 69, 969, 205, 209, 216, 474, 96, 97, 98, 100, 357, 486, 746, 238, 751}
{1}
set()
{512, 191, 354, 70, 199, 168, 491, 1101, 367, 240, 1106, 311, 249, 381, 255}
{1}
set()
{2, 515, 516, 4, 7, 520, 8, 9, 11, 12, 10, 17, 531, 22, 28, 31, 546, 38, 1063, 559, 50, 52, 566, 568, 56, 58, 569, 64, 69, 70, 582, 72, 588, 77, 79, 591, 81, 82, 83, 87, 89, 603, 91, 95, 96, 97, 98, 1133, 111, 625, 1141, 633, 636, 132, 135, 650, 651, 141, 143, 144, 655, 662, 663, 153, 154, 156, 160, 161, 162, 679, 168, 172, 173, 

set()
{515, 902, 523, 660, 286, 169, 431, 50, 181, 310, 191, 194, 582, 199, 204, 212, 214, 474, 357, 628, 250}
{1}
{1}
{1, 258, 131, 269, 275, 405, 1177, 538, 283, 300, 50, 181, 311, 328, 333, 597, 222, 740, 229, 873, 748, 111, 369, 755, 117, 761, 127}
{1}
set()
{1226, 333, 1136, 275, 470, 88, 319}
{1}
set()
{258, 269, 289, 301, 302, 303, 312, 313, 315, 316, 319, 321, 328, 332, 333, 340, 345, 347, 354, 751}
{1}
set()
{512, 640, 515, 10, 638, 652, 923, 30, 165, 166, 170, 171, 179, 52, 190, 707, 70, 713, 970, 971, 462, 207, 83, 212, 213, 1622, 855, 86, 730, 1396, 1018, 509, 382, 639}
{1}
{1}
{1, 7, 8, 12, 15, 1039, 531, 1044, 21, 22, 1047, 25, 28, 29, 542, 31, 1053, 1061, 550, 40, 553, 42, 554, 559, 561, 50, 51, 1074, 1078, 56, 568, 570, 63, 66, 1091, 68, 578, 71, 584, 73, 1098, 72, 588, 79, 82, 83, 597, 85, 88, 602, 91, 95, 99, 107, 110, 623, 625, 117, 118, 121, 125, 638, 131, 132, 133, 645, 142, 143, 144, 655, 658, 660, 151, 154, 155, 161, 162, 164, 168, 684, 173, 174, 172, 178, 692, 1

Мы видим, что качество просело, это объясняется тем, что мы начали рекомендовать из всего пула фильмов (не все видело наше ранжирование), данное упражнение должно было показать вам что такое многостадийные рекомендации. Вы должны усвоить, что обе стадии - ранжирование и кандидатогенерация тесно связаны между собой и очень важны каждая сама по себе.